# Ejercicio 1b: Estimación del Número de Partículas

**Estudiantes:**
- Sergio Andrés Díaz Vera (seadiazve@unal.edu.co)
- Julián Mateo Espinosa Ospina (juespinosao@unal.edu.co)

**Curso:** Cadenas de Markov y Aplicaciones (2025-II)

## Descripción

Análisis sistemático del número típico de partículas en configuraciones Hard-Core para diferentes tamaños de rejilla.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Agregar ruta al directorio padre para importar módulos
sys.path.append(os.path.join(os.path.dirname(os.path.abspath('')), '..'))

from src.hard_core import gibbs_sampler_hard_core, contar_particulas
from src.estadisticas import analizar_multiple_K, crear_tabla_estadisticas
from src.visualizacion import graficar_escalamiento

## 1. Análisis para múltiples tamaños K

Analizamos cómo varía el número de partículas con el tamaño de la rejilla.

In [ ]:
# Parámetros
K_valores = [3, 5, 7, 10, 12, 15, 20]
T = 10000
n_muestras = 100

# Realizar análisis
print("Analizando diferentes tamaños de rejilla...")
print("Esto puede tomar algunos minutos...\n")

resultados = analizar_multiple_K(K_valores, T, n_muestras)

# Mostrar tabla de resultados
print(crear_tabla_estadisticas(resultados))

## 2. Análisis de escalamiento

Graficamos el número medio de partículas vs K² para analizar la relación de escalamiento.

In [ ]:
# Extraer medias
medias = [resultados[K]['estadisticas']['media'] for K in K_valores]
densidades = [resultados[K]['estadisticas']['media'] / (K**2) for K in K_valores]

# Gráfico de escalamiento
graficar_escalamiento(K_valores, medias)
plt.show()

# Densidad promedio
print(f"\nDensidad promedio (ρ = μ/K²):")
for K, rho in zip(K_valores, densidades):
    print(f"K={K:2d}: ρ = {rho:.4f}")

print(f"\nDensidad límite estimada: ρ_∞ ≈ {np.mean(densidades[3:]):.4f}")

## 3. Histogramas por tamaño K

Visualizamos la distribución del número de partículas para cada tamaño.

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for idx, K in enumerate(K_valores):
    particulas = resultados[K]['particulas']
    stats = resultados[K]['estadisticas']
    
    axes[idx].hist(particulas, bins=20, edgecolor='black', alpha=0.7)
    axes[idx].axvline(stats['media'], color='r', linestyle='--', 
                     label=f"μ={stats['media']:.1f}")
    axes[idx].set_title(f"K={K}")
    axes[idx].set_xlabel('Número de partículas')
    axes[idx].set_ylabel('Frecuencia')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

axes[-1].axis('off')
plt.tight_layout()
plt.show()

## 4. Análisis de convergencia detallado

Analizamos la convergencia variando el tiempo T para un K fijo.

In [ ]:
K = 10
T_valores = [100, 500, 1000, 5000, 10000, 50000, 100000]
n_muestras = 100

medias_T = []
stds_T = []

for T in T_valores:
    particulas = []
    for i in range(n_muestras):
        config = gibbs_sampler_hard_core(K, T, semilla=i)
        particulas.append(contar_particulas(config))
    
    medias_T.append(np.mean(particulas))
    stds_T.append(np.std(particulas))

# Tabla de convergencia
print("Análisis de convergencia para K=10:\n")
print("T\t\tMedia\tStd\tΔ_t (%)")
print("-" * 50)
for i, T in enumerate(T_valores):
    if i == 0:
        delta = "---"
    else:
        delta = f"{100*abs(medias_T[i]-medias_T[i-1])/medias_T[i-1]:.1f}"
    print(f"{T}\t\t{medias_T[i]:.2f}\t{stds_T[i]:.2f}\t{delta}")

# Gráfico de convergencia
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.semilogx(T_valores, medias_T, 'o-', linewidth=2, markersize=8)
ax1.set_xlabel('T (iteraciones)')
ax1.set_ylabel('Número medio de partículas')
ax1.set_title(f'Convergencia de μ (K={K})')
ax1.grid(True, alpha=0.3)

ax2.semilogx(T_valores, stds_T, 'o-', color='orange', linewidth=2, markersize=8)
ax2.set_xlabel('T (iteraciones)')
ax2.set_ylabel('Desviación estándar')
ax2.set_title(f'Convergencia de σ (K={K})')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()